<a href="https://colab.research.google.com/github/janimiyarj/MIS-515-Practise/blob/main/Copy_of_student_placement_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('students_placement.csv')

In [ ]:
df.shape

(300, 4)

In [ ]:
df.sample(5)

,cgpa,iq,profile_score,placed
72,6.15,85,63,1
214,5.80,107,92,0
114,5.89,108,72,0
64,7.50,102,59,0
105,7.35,93,45,1


In [ ]:
X = df.drop(columns=['placed'])
y = df['placed']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
scaler = StandardScaler()
X_train_trf = scaler.fit_transform(X_train)
X_test_trf = scaler.transform(X_test)
accuracy_score(y_test,
               LogisticRegression()
               .fit(X_train_trf,y_train)
               .predict(X_test_trf))

0.8833333333333333

In [ ]:
from sklearn.ensemble import RandomForestClassifier
accuracy_score(y_test,RandomForestClassifier().fit(X_train,y_train).predict(X_test))

0.9

In [ ]:
from sklearn.svm import SVC
accuracy_score(y_test,SVC(kernel='rbf').fit(X_train,y_train).predict(X_test))

0.9333333333333333

In [ ]:
svc = SVC(kernel='rbf')
svc.fit(X_train,y_train)

SVC()

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier()

In [ ]:
import pickle
pickle.dump(svc,open('model.pkl','wb'))

In [ ]:
import pickle
pickle.dump(rf,open('model1.pkl','wb'))

In [ ]:
svc.predict(np.array([0,0,0]).reshape(1,3))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


array([1])

In [ ]:
rf.predict(np.array([0,0,0]).reshape(1,3))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array([0])

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip


--2023-10-28 01:37:54--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.28M  58.4MB/s    in 0.2s    

2023-10-28 01:37:55 (58.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13921656/13921656]



In [ ]:
!unzip ngrok-stable-linux-amd64.zip


Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
!chmod +x ngrok
!sudo mv ngrok /usr/local/bin/


In [ ]:
!ngrok authtoken 2XK47zD7jVqVgVeY4vJ7IfdpT2X_5q81CHEcBmyBCfqbeURio


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install flask-ngrok

In [ ]:
import os
os.makedirs("templates", exist_ok=True)

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request
import pickle
import numpy as np

model = pickle.load(open('model1.pkl', 'rb'))
app = Flask(__name__)
run_with_ngrok(app)


@app.route('/')
@app.route('/home/')
def home():
  return render_template('Boot.html')

@app.route('/predict', methods=['GET', 'POST'])
def input_form():
    result = None

    if request.method == 'POST':
        cgpa = float(request.form.get('cgpa'))
        iq = int(request.form.get('iq'))
        profile_score = int(request.form.get('profile_score'))

        # prediction
        result = model.predict(np.array([cgpa, iq, profile_score]).reshape(1, 3))
        if result[0] == 1:
          result = 'Congratulations, you have high chance of getting placement'
        else:
          result = 'Difficult to get placement'

    return render_template('boot2.html', result=result)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://502b-35-196-18-66.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [28/Oct/2023 03:36:47] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Oct/2023 03:36:48] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [28/Oct/2023 03:36:54] "POST /predict HTTP/1.1" 200 -


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,render_template,request
import pickle
import numpy as np

model = pickle.load(open('model.pkl','rb'))
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def hello():
  return """ <h1>Input Form</h1>
    <form action="/predict" method="POST">
        <label for="cgpa">CGPA:</label>
        <input type="text" id="cgpa" name="cgpa" required><br><br>

        <label for="iq">IQ:</label>
        <input type="text" id="iq" name="iq" required><br><br>

        <label for="profile-score">Profile Score:</label>
        <input type="text" id="profile_score" name="profile_score" required><br><br>

        <input type="submit" value="Submit">
    </form> """
# return render_template('/content/firstwebhtml.html')
#def index():
#    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict_placement():
    cgpa = float(request.form.get('cgpa'))
    iq = int(request.form.get('iq'))
    profile_score = int(request.form.get('profile_score'))

# prediction
    result = model.predict(np.array([cgpa,iq,profile_score]).reshape(1,3))
    return """ <h1>Input Form</h1>
    <form action="/predict" method="POST">
        <label for="cgpa">CGPA:</label>
        <input type="text" id="cgpa" name="cgpa" required><br><br>

        <label for="iq">IQ:</label>
        <input type="text" id="iq" name="iq" required><br><br>

        <label for="profile-score">Profile Score:</label>
        <input type="text" id="profile_score" name="profile_score" required><br><br>

        <input type="submit" value="Submit"><br><br>
        {% if result[0] %}
          <p>{{Your Status of placement is Placed}}</p>
        {% else %}
          <p>{{Not placed}}</p>
        {% endif %}
    </form> """


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://940e-34-80-132-255.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [28/Oct/2023 01:29:58] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [28/Oct/2023 01:29:58] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [28/Oct/2023 01:30:07] "POST /predict HTTP/1.1" 200 -


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,render_template,request
import pickle
import numpy as np

model = pickle.load(open('model.pkl','rb'))

In [ ]:
model

SVC()

In [ ]:
cgpa = 7.64
iq = 102
profile_score = 51
# prediction
result = model.predict(np.array([cgpa, iq, profile_score]).reshape(1, 3))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
